In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import sys
import os

In [2]:
def Mixed_matrix(M):
    L = int(np.sqrt(len(M)))
    M_sqrt = np.reshape(M, (L, L))
    for i in range(L):
        j, k = np.random.randint(0, L, 2)
        if np.random.rand() < 0.5:
            temp = np.copy(M_sqrt[:,j])
            M_sqrt[:,j] = M_sqrt[:,k]
            M_sqrt[:,k] = temp
        else:
            temp = np.copy(M_sqrt[j,:])
            M_sqrt[j,:] = M_sqrt[k,:]
            M_sqrt[k,:] = temp
    return np.reshape(M_sqrt,(1,L*L))[0]

In [3]:
def get_mag(M):
    return np.mean(M)

def get_E(M):
    return np.mean(-0.5*M*(np.roll(M,1,axis=0)+np.roll(M,-1,axis=0)+np.roll(M,1,axis=1)+np.roll(M,-1,axis=1)))

def get_E2(M):
    return np.mean((-0.5*M*(np.roll(M,1,axis=0)+np.roll(M,-1,axis=0)+np.roll(M,1,axis=1)+np.roll(M,-1,axis=1)))**2)

def get_Cv(M, T):
    return (get_E2(M)-get_E(M)**2)/T**2

def get_Cm(M, T):
    return (get_mag(M*M)-get_mag(M)**2)/T

In [28]:
path = "../../Generate_Data/Ising_Model/DATA/"

Size_Folders = np.array(glob.glob(path+"L*"))
Size_Folders = Size_Folders[np.argsort([int(s.split("L")[-1]) for s in Size_Folders])]

for folder_size in Size_Folders:
    if not os.path.exists(folder_size.replace('/DATA/', '/DATA_Control/')): 
        os.mkdir(folder_size.replace('/DATA/', '/DATA_Control/'))

    L_folders = np.sort(glob.glob(folder_size+"/Ising*"))
    for l_folder in L_folders:
        #print(l_folder)
        new_l_folder = l_folder.replace('/DATA/', '/DATA_Control/')
        #print(new_l_folder)
        if not os.path.exists(new_l_folder): 
            os.mkdir(new_l_folder)
            os.mkdir(new_l_folder+"/DATA/")
            
        Matrix_T = np.sort(glob.glob(l_folder+"/DATA/Matrix_configurations/T*.csv"))
        Temperatures = np.array([float(m_t.split('/T')[-1].split(".csv")[0]) for m_t in Matrix_T])
        Configurations = np.array([pd.read_csv(m_t, sep=" ", header=None).values for m_t in Matrix_T])
        M_, E_, Cv_, Cm_ = [], [], [], []        
        for i in range(len(Matrix_T)): #for each T
            L = int(np.sqrt(Configurations[i].shape[1]))
            M_.append(np.abs(get_mag(Configurations[i].reshape((L,L)))))
            E_.append(get_E(Configurations[i].reshape((L,L))))
            Cv_.append(get_Cv(Configurations[i].reshape((L,L)), Temperatures[i]))
            Cm_.append(get_Cm(Configurations[i].reshape((L,L)), Temperatures[i]))
        
        data_csv = pd.DataFrame(np.array([Temperatures,E_,M_,Cv_,Cm_]).transpose(), 
                                columns=['Temperature','Energy','Magnetization','Cv','Chi_m'])
        
        data_csv.to_csv(new_l_folder+"/DATA/data.csv", index=None)

In [29]:
path = "../../Generate_Data/Ising_Model/DATA/"

Size_Folders = np.array(glob.glob(path+"L*"))
Size_Folders = Size_Folders[np.argsort([int(s.split("L")[-1]) for s in Size_Folders])]

for folder_size in Size_Folders:
    if not os.path.exists(folder_size.replace('/DATA/', '/DATA_Random/')): 
        os.mkdir(folder_size.replace('/DATA/', '/DATA_Random/'))

    L_folders = np.sort(glob.glob(folder_size+"/Ising*"))
    for l_folder in L_folders:
        #print(l_folder)
        new_l_folder = l_folder.replace('/DATA/', '/DATA_Random/')
        #print(new_l_folder)
        if not os.path.exists(new_l_folder): 
            os.mkdir(new_l_folder)
            os.mkdir(new_l_folder+"/DATA/")
            
        Matrix_T = np.sort(glob.glob(l_folder+"/DATA/Matrix_configurations/T*.csv"))
        Temperatures = np.array([float(m_t.split('/T')[-1].split(".csv")[0]) for m_t in Matrix_T])
        Configurations = np.array([pd.read_csv(m_t, sep=" ", header=None).values for m_t in Matrix_T])
        M_, E_, Cv_, Cm_ = [], [], [], []        
        for i in range(len(Matrix_T)): #for each T
            Configurations_random = Mixed_matrix(Configurations[i][0])
            L = int(np.sqrt(Configurations_random.shape[0]))
            M_.append(np.abs(get_mag(Configurations_random.reshape((L,L)))))
            E_.append(get_E(Configurations_random.reshape((L,L))))
            Cv_.append(get_Cv(Configurations_random.reshape((L,L)), Temperatures[i]))
            Cm_.append(get_Cm(Configurations_random.reshape((L,L)), Temperatures[i]))
        
        data_csv = pd.DataFrame(np.array([Temperatures,E_,M_,Cv_,Cm_]).transpose(), 
                                columns=['Temperature','Energy','Magnetization','Cv','Chi_m'])
        
        data_csv.to_csv(new_l_folder+"/DATA/data.csv", index=None)